In [1]:

from key import KEY as OPENAI_API_KEY
from key import SS_KEY as SS_API_KEY
from urllib.request import Request, urlopen

import json
import os
import pickle
import time
import requests
from bs4 import BeautifulSoup

# With Semantic Scholar

In [2]:
# Directly define the API key (Reminder: Securely handle API keys in production environments)
api_key = SS_API_KEY

# Define headers with API key
headers = {'x-api-key': api_key}

In [33]:
def get_paper_data(paper_id):
  url = 'https://api.semanticscholar.org/graph/v1/paper/' + paper_id

  # Define which details about the paper you would like to receive in the response
  paper_data_query_params = {'fields': 'title,citationCount'}

  # Send the API request and store the response in a variable
  response = requests.get(url, params=paper_data_query_params)
  if response.status_code == 200:
    return response.json()
  else:
    return None

def download_file(url, filename):
  try:
    response = requests.get(url=url, headers={'User-Agent': 'Mozilla/5.0'})

    with open(filename, 'wb') as file:
      file.write(response.content)
    print(f"File downloaded successfully: {filename}")
  
  except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")




In [34]:
# Define the API endpoint URL
url = 'https://api.semanticscholar.org/graph/v1/paper/search'

# More specific query parameter
query_params = {'query': 'Large-scale simultaneous measurement of epitopes and transcriptomes in single cells', }

# Send the API request
search_response = requests.get(url, params=query_params, headers=headers)

# Check if the request was successful (status code 200)
if search_response.status_code == 200:
  search_response = search_response.json()

  # Retrieve the paper id corresponding to the 1st result in the list
  paper_id = search_response['data'][0]['paperId']

  # Retrieve the paper details corresponding to this paper id using the function we defined earlier.
  paper_details = get_paper_data(paper_id)
  print(paper_details)
else:
  # Handle potential errors or non-200 responses
  print(f"Relevance Search Request failed with status code {search_response.status_code}: {search_response.text}")


{'paperId': 'e6ac2e5fdeb3f2d94efeeee5639c06cba2248f4e', 'title': 'Large-scale simultaneous measurement of epitopes and transcriptomes in single cells', 'citationCount': 1713}


In [64]:
# Define the directory name
out_dir = 'citing_paper_abstracts'

# Create the directory if it doesn't exist
if not os.path.exists(out_dir):
    os.makedirs(out_dir)


In [70]:
# Getting ALL the citations
i = 0
total_citations = paper_details['citationCount']
while (i < total_citations):
    try:
        query = 'https://api.semanticscholar.org/graph/v1/paper/' + paper_details['paperId'] + '/' + 'citations?' + f'fields=abstract,title&offset={i}&limit=1'
        print(query)
        cites = requests.get(query)
        
        title = cites.json()['data'][0]['citingPaper']['title'].replace('/', '') + '.json'
        json_data = json.dumps(cites.json()['data'][0], indent=4)
        
        # Write the JSON string to a file
        outpath = os.path.join(out_dir, title)
        with open(outpath, 'w') as file:
            file.write(json_data)
        print(json_data)
        time.sleep(3)
    except:
        continue
    
    finally:
        i+=1

https://api.semanticscholar.org/graph/v1/paper/e6ac2e5fdeb3f2d94efeeee5639c06cba2248f4e/citations?fields=abstract,title&offset=0&limit=1
{
    "citingPaper": {
        "paperId": "a83dbce941baa359f9c950fa30e353ef39f9d564",
        "title": "Single-cell genomics-based immune and disease monitoring in blood malignancies",
        "abstract": "Achieving long-term disease control using therapeutic immunomodulation is a long-standing concept with a strong tradition in blood malignancies. Besides allogeneic hematopoietic stem cell transplantation that continues to provide potentially curative treatment for otherwise challenging diagnoses, recent years have seen impressive progress in immunotherapies for leukemias and lymphomas with immune checkpoint blockade, bispecific monoclonal antibodies, and CAR T cell therapies. Despite their success, non-response, relapse, and immune toxicities remain frequent, thus prioritizing the elucidation of the underlying mechanisms and identifying predictive b

In [5]:
# if paper_details:
#     if paper_details['isOpenAccess']:
#         pdf_url = paper_details['openAccessPdf']['url'] 
#         print(pdf_url)


https://europepmc.org/articles/pmc5669064?pdf=render


In [6]:
# download_file(pdf_url, "test.pdf")

File downloaded successfully: test.pdf
